In [2]:
## Next steps:
## Add more data
## Make better model
## Gridsearch

In [80]:
from sklearn import tree #decision tree
from sklearn.model_selection import cross_val_score # evaluate our model
from sklearn.model_selection import GridSearchCV # gridsearch to find best hyper parameters
import pandas as pd
import numpy as np

In [31]:
### Data Exploration
np.random.seed(1)
df = pd.read_csv('2020.csv', header=[0,1])
df.head(2)

Unnamed: 0_level_0 Unnamed: 1_level_0 Overall                             \
                  Rk             School       G   W   L   W-L%   SRS   SOS   
0                  1  Abilene Christian      23  13  10  0.565 -2.90 -5.05   
1                  2          Air Force      25   9  16  0.360 -0.53  2.88   

  Conf.      ... School Totals                                                
      W   L  ...            FT  FTA    FT%  ORB  TRB  AST  STL BLK  TOV   PF  
0     8   4  ...           366  496  0.738  262  791  344  212  60  328  485  
1     3  10  ...           387  519  0.746  178  810  354  124  39  305  424  

[2 rows x 34 columns]

In [32]:
### Data Cleaning

### Only want a single header
df = pd.read_csv('2020.csv', header=[1])
df.head(2)

,Rk,School,G,W,L,W-L%,SRS,SOS,W.1,L.1,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,1,Abilene Christian,23,13,10,0.565,-2.90,-5.05,8,4,...,366,496,0.738,262,791,344,212,60,328,485
1,2,Air Force,25,9,16,0.360,-0.53,2.88,3,10,...,387,519,0.746,178,810,354,124,39,305,424


In [33]:
#rename w.1 etc
df.rename(columns={"W.1": "Conf W", "W.2": "Home W", "W.3": "Away W"}, inplace=True)
df.rename(columns={"L.1": "Conf L", "L.2": "Home L", "L.3": "Away L"}, inplace=True)
df.head(1)

,Rk,School,G,W,L,W-L%,SRS,SOS,Conf W,Conf L,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,1,Abilene Christian,23,13,10,0.565,-2.9,-5.05,8,4,...,366,496,0.738,262,791,344,212,60,328,485


In [34]:
df.drop(['Unnamed: 16', 'Rk'], axis=1, inplace = True)
df.head(4)

,School,G,W,L,W-L%,SRS,SOS,Conf W,Conf L,Home W,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,Abilene Christian,23,13,10,0.565,-2.90,-5.05,8,4,10,...,366,496,0.738,262,791,344,212,60,328,485
1,Air Force,25,9,16,0.360,-0.53,2.88,3,10,6,...,387,519,0.746,178,810,354,124,39,305,424
2,Akron,24,18,6,0.750,7.97,-0.53,8,3,12,...,358,461,0.777,236,896,329,117,68,320,421
3,Alabama A&M,22,6,16,0.273,-18.22,-8.27,3,8,4,...,271,388,0.698,217,757,238,132,44,258,387


In [35]:
### Data Exploration of new table
schedule_df_full = pd.read_csv('big_ten.csv')
schedule_df_full[schedule_df_full['PTS'].isnull()==False].head()

,Date,Visitor/Neutral,PTS,Home/Neutral,PTS.1,OT,Notes
0,Fri Dec 6 2019,Iowa,91.0,Michigan,103.0,NaN,NaN
1,Sat Dec 7 2019,Illinois,58.0,Maryland,59.0,NaN,NaN
2,Sat Dec 7 2019,Penn State,74.0,Ohio State,106.0,NaN,NaN
3,Sat Dec 7 2019,Indiana,64.0,Wisconsin,84.0,NaN,NaN
4,Sun Dec 8 2019,Rutgers,65.0,Michigan State,77.0,NaN,NaN


In [36]:
schedule_df_full[schedule_df_full['Notes'].isnull()==False].head()

,Date,Visitor/Neutral,PTS,Home/Neutral,PTS.1,OT,Notes


In [37]:
### Drop unneeded columns  My simple model leave out OT for now.
schedule_df_full.drop(['Date','Notes', 'OT'], axis=1, inplace=True)

In [38]:
schedule_df_full.tail()

,Visitor/Neutral,PTS,Home/Neutral,PTS.1
135,Rutgers,NaN,Purdue,NaN
136,Iowa,NaN,Illinois,NaN
137,Michigan,NaN,Maryland,NaN
138,Ohio State,NaN,Michigan State,NaN
139,Nebraska,NaN,Minnesota,NaN


In [39]:
#Remove unplayed games
schedule_df = schedule_df_full[schedule_df_full['PTS'].isnull()==False]

In [40]:
schedule_df.head()

,Visitor/Neutral,PTS,Home/Neutral,PTS.1
0,Iowa,91.0,Michigan,103.0
1,Illinois,58.0,Maryland,59.0
2,Penn State,74.0,Ohio State,106.0
3,Indiana,64.0,Wisconsin,84.0
4,Rutgers,65.0,Michigan State,77.0


In [41]:

combine = pd.merge(schedule_df,df, left_on='Visitor/Neutral', right_on='School')
combine.head()

,Visitor/Neutral,PTS,Home/Neutral,PTS.1,School,G,W,L,W-L%,SRS,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,Iowa,91.0,Michigan,103.0,Iowa,24,17,7,0.708,16.55,...,377,505,0.747,289,916,426,157,87,286,372
1,Iowa,86.0,Penn State,89.0,Iowa,24,17,7,0.708,16.55,...,377,505,0.747,289,916,426,157,87,286,372
2,Iowa,70.0,Nebraska,76.0,Iowa,24,17,7,0.708,16.55,...,377,505,0.747,289,916,426,157,87,286,372
3,Iowa,75.0,Northwestern,62.0,Iowa,24,17,7,0.708,16.55,...,377,505,0.747,289,916,426,157,87,286,372
4,Iowa,72.0,Maryland,82.0,Iowa,24,17,7,0.708,16.55,...,377,505,0.747,289,916,426,157,87,286,372


In [42]:
Schedule_and_stats = pd.merge(combine, df, left_on = 'Home/Neutral', right_on = 'School')
Schedule_and_stats.head()

,Visitor/Neutral,PTS,Home/Neutral,PTS.1,School_x,G_x,W_x,L_x,W-L%_x,SRS_x,...,FT_y,FTA_y,FT%_y,ORB_y,TRB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y
0,Iowa,91.0,Michigan,103.0,Iowa,24,17,7,0.708,16.55,...,259,363,0.713,210,845,343,127,109,266,347
1,Illinois,64.0,Michigan,62.0,Illinois,24,16,8,0.667,15.80,...,259,363,0.713,210,845,343,127,109,266,347
2,Penn State,72.0,Michigan,63.0,Penn State,24,19,5,0.792,18.01,...,259,363,0.713,210,845,343,127,109,266,347
3,Rutgers,63.0,Michigan,69.0,Rutgers,24,17,7,0.708,15.56,...,259,363,0.713,210,845,343,127,109,266,347
4,Ohio State,61.0,Michigan,58.0,Ohio State,23,15,8,0.652,18.50,...,259,363,0.713,210,845,343,127,109,266,347


In [43]:
X = Schedule_and_stats.drop(["Visitor/Neutral","Home/Neutral","School_x","School_y","PTS","PTS.1"], axis=1)
Y = Schedule_and_stats[["PTS","PTS.1"]]

In [103]:
reg = tree.DecisionTreeRegressor(random_state=0)
reg = reg.fit(X, Y)

In [104]:
combine_full = pd.merge(schedule_df_full,df, left_on='Visitor/Neutral', right_on='School')
Schedule_and_stats_full = pd.merge(combine_full, df, left_on = 'Home/Neutral', right_on = 'School')

In [105]:
test = Schedule_and_stats_full[(Schedule_and_stats_full["Visitor/Neutral"] =="Michigan") & (Schedule_and_stats_full["Home/Neutral"] =="Rutgers")]
test


,Visitor/Neutral,PTS,Home/Neutral,PTS.1,School_x,G_x,W_x,L_x,W-L%_x,SRS_x,...,FT_y,FTA_y,FT%_y,ORB_y,TRB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y
126,Michigan,NaN,Rutgers,NaN,Michigan,23,14,9,0.609,17.15,...,285,443,0.643,299,988,316,162,114,305,390


In [106]:
test = test.drop(["Visitor/Neutral","Home/Neutral","School_x","School_y","PTS","PTS.1"], axis=1)

In [107]:
reg.predict(test)
#First prediction: Michigan 62, Rutgers 71

array([[83., 90.]])

In [108]:
### cross validations
print(cross_val_score(reg, X, Y, cv=5))

#First model performs HORRIBLY!

[-0.93425194 -1.05742786 -0.83916977 -1.05796534 -1.25172123]


//anaconda3/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "mul

In [94]:
# use grid search to determine decision tree depth
params = {"max_depth":range(1,10)}
treereg = tree.DecisionTreeRegressor(random_state=0)
reg2 = GridSearchCV(estimator = treereg, param_grid=params)
reg2.fit(X,Y)
reg2.best_params_


//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent wit

{'max_depth': 1}

In [100]:
means = reg2.cv_results_['mean_test_score']
stds = reg2.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, reg2.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()


-0.017 (+/-0.249) for {'max_depth': 1}

-0.207 (+/-0.369) for {'max_depth': 2}

-0.286 (+/-0.359) for {'max_depth': 3}

-0.452 (+/-0.473) for {'max_depth': 4}

-0.662 (+/-0.432) for {'max_depth': 5}

-0.670 (+/-0.306) for {'max_depth': 6}

-0.921 (+/-0.142) for {'max_depth': 7}

-1.084 (+/-0.378) for {'max_depth': 8}

-1.119 (+/-0.485) for {'max_depth': 9}

